# 🔄 Workflows Básicos no Agno: Orquestração Inteligente de Agentes

Eaí pessoal! Pedro Guth aqui novamente! 🚀

No módulo anterior a gente viu como fazer **Teams** - agentes colaborando simultaneamente. Agora vamos dar um passo além e aprender sobre **Workflows** - que é tipo uma receita de bolo, mas para agentes! 

Tá, mas o que é um Workflow? É basicamente um fluxo de trabalho onde você define **exatamente** quais passos seguir e em que ordem. É como se você fosse o diretor de um filme e cada agente é um ator que entra em cena na hora certa!

## 🎯 O que vamos aprender hoje:

1. **Conceitos fundamentais** de Workflows
2. **Diferenças** entre Teams e Workflows
3. **Implementação prática** de fluxos sequenciais
4. **Steps simples** e como conectá-los
5. **Casos de uso reais** com exemplos

Bora mergulhar nesse mundo dos workflows! 🏊‍♂️

## 🛠️ Setup Inicial - Preparando o Ambiente

Primeiro, vamos instalar e configurar tudo que precisamos. Lembra da nossa configuração dos módulos anteriores? Vamos usar a mesma base!

In [ ]:
# Instalação das dependências
!pip install -q agno google-generativeai matplotlib seaborn

In [ ]:
# Imports necessários
import os
from getpass import getpass
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

# Imports do Agno para Workflows
from agno.workflows import Workflow, Step
from agno.agent import Agent
from agno.models.google import Gemini
from agno.tools.duckduckgo import DuckDuckGoTools
from agno.tools.hackernews import HackerNewsTools

print("📦 Imports realizados com sucesso!")

In [ ]:
# Configuração da API Key do Google
# Cole sua chave do Google AI Studio aqui
GOOGLE_API_KEY = getpass("Digite sua Google API Key: ")
os.environ['GOOGLE_API_KEY'] = GOOGLE_API_KEY

print("🔑 API Key configurada!")

## 🧠 Conceitos Fundamentais: O que são Workflows?

Vamos começar com uma analogia massa: imagina que você quer fazer um **sanduíche gourmet**. 

- **Team**: Todo mundo na cozinha fazendo tudo ao mesmo tempo (caos organizado!)
- **Workflow**: Uma receita step-by-step onde cada pessoa faz sua parte na hora certa

### 📋 Características dos Workflows:

1. **Sequencial**: Um passo após o outro
2. **Determinístico**: Sempre segue a mesma ordem
3. **Controlado**: Você define exatamente o que acontece
4. **Modular**: Cada step é independente

### 🆚 Teams vs Workflows:

| Aspecto | Teams | Workflows |
|---------|-------|----------|
| **Execução** | Paralela/Simultânea | Sequencial |
| **Controle** | Agente líder decide | Você define os passos |
| **Flexibilidade** | Mais adaptativo | Mais previsível |
| **Uso ideal** | Brainstorming, discussões | Processos definidos |

**Dica!** Use Workflows quando você sabe exatamente que passos seguir e Teams quando quer que os agentes colaborem de forma mais livre!

## 🏗️ Arquitetura dos Workflows no Agno

Vamos visualizar como funciona a estrutura de um Workflow:

In [ ]:
# Vamos criar um diagrama conceitual da arquitetura
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from matplotlib.patches import FancyBboxPatch, Rectangle

fig, ax = plt.subplots(figsize=(12, 8))

# Configurar o plot
ax.set_xlim(0, 10)
ax.set_ylim(0, 8)
ax.axis('off')

# Título
ax.text(5, 7.5, 'Arquitetura de Workflows no Agno', 
        fontsize=16, fontweight='bold', ha='center')

# Workflow principal
workflow_box = FancyBboxPatch((1, 5.5), 8, 1.5, 
                             boxstyle="round,pad=0.1", 
                             facecolor='lightblue', 
                             edgecolor='blue', linewidth=2)
ax.add_patch(workflow_box)
ax.text(5, 6.2, 'WORKFLOW', fontsize=14, fontweight='bold', ha='center')
ax.text(5, 5.8, 'Orquestrador dos Steps', fontsize=10, ha='center')

# Steps
step_colors = ['lightgreen', 'lightyellow', 'lightcoral']
step_names = ['Step 1\nPesquisa', 'Step 2\nAnálise', 'Step 3\nRelatório']

for i, (color, name) in enumerate(zip(step_colors, step_names)):
    x_pos = 1.5 + i * 2.5
    step_box = FancyBboxPatch((x_pos, 3), 2, 1.5, 
                             boxstyle="round,pad=0.1", 
                             facecolor=color, 
                             edgecolor='gray', linewidth=1)
    ax.add_patch(step_box)
    ax.text(x_pos + 1, 3.75, name, fontsize=10, fontweight='bold', ha='center')
    
    # Agente dentro do step
    agent_box = Rectangle((x_pos + 0.2, 1), 1.6, 1, 
                         facecolor='white', 
                         edgecolor='black', linewidth=1)
    ax.add_patch(agent_box)
    ax.text(x_pos + 1, 1.5, f'Agente {i+1}', fontsize=9, ha='center')

# Setas conectando os steps
for i in range(2):
    ax.arrow(3.5 + i * 2.5, 3.75, 1.5, 0, 
             head_width=0.15, head_length=0.2, 
             fc='black', ec='black')

# Setas conectando workflow aos steps
for i in range(3):
    ax.arrow(2.5 + i * 2.5, 5.5, 0, -0.8, 
             head_width=0.15, head_length=0.1, 
             fc='blue', ec='blue', alpha=0.7)

# Legenda
ax.text(0.5, 0.5, '💡 Fluxo: Input → Step 1 → Step 2 → Step 3 → Output', 
        fontsize=11, fontweight='bold')

plt.tight_layout()
plt.show()

print("🎨 Arquitetura visualizada!")

## 🚀 Nosso Primeiro Workflow: Pesquisa de Tecnologia

Vamos criar um workflow simples que:
1. **Pesquisa** informações sobre uma tecnologia
2. **Analisa** as informações encontradas
3. **Gera** um relatório final

É como ter três especialistas trabalhando em sequência!

In [ ]:
# Primeiro, vamos criar nossos agentes especialistas

# 🔍 Agente Pesquisador
pesquisador = Agent(
    name="Pesquisador Tech",
    model=Gemini(id="gemini-2.0-flash"),
    role="Especialista em pesquisa de tecnologias",
    instructions=[
        "Você é um pesquisador experiente",
        "Busque informações atuais e relevantes",
        "Foque em dados técnicos e tendências",
        "Seja objetivo e preciso"
    ],
    tools=[DuckDuckGoTools()],
    markdown=True
)

# 📊 Agente Analista
analista = Agent(
    name="Analista Tech",
    model=Gemini(id="gemini-2.0-flash"),
    role="Especialista em análise tecnológica",
    instructions=[
        "Você é um analista técnico experiente",
        "Analise dados de forma crítica",
        "Identifique pontos fortes e fracos",
        "Faça comparações quando relevante"
    ],
    markdown=True
)

# 📝 Agente Redator
redator = Agent(
    name="Redator Tech",
    model=Gemini(id="gemini-2.0-flash"),
    role="Especialista em comunicação técnica",
    instructions=[
        "Você é um redator técnico especializado",
        "Crie relatórios claros e bem estruturados",
        "Use linguagem acessível mas precisa",
        "Organize as informações de forma lógica"
    ],
    markdown=True
)

print("👥 Agentes criados:")
print(f"🔍 {pesquisador.name}")
print(f"📊 {analista.name}")
print(f"📝 {redator.name}")

## 🔄 Criando os Steps do Workflow

Agora vamos definir cada passo do nosso workflow. Cada **Step** é como uma estação de trabalho com um agente especializado:

In [ ]:
# 🔄 Step 1: Pesquisa
step_pesquisa = Step(
    name="pesquisa_tecnologia",
    description="Pesquisa informações sobre a tecnologia solicitada",
    agent=pesquisador,
    # O step vai receber o input do usuário
    inputs=["tecnologia"],
    # E vai gerar um output chamado "dados_pesquisa"
    outputs=["dados_pesquisa"]
)

# 📊 Step 2: Análise
step_analise = Step(
    name="analise_dados",
    description="Analisa os dados coletados na pesquisa",
    agent=analista,
    # Recebe os dados da pesquisa
    inputs=["dados_pesquisa"],
    # Gera uma análise
    outputs=["analise_tecnica"]
)

# 📝 Step 3: Relatório
step_relatorio = Step(
    name="gerar_relatorio",
    description="Gera relatório final baseado na análise",
    agent=redator,
    # Recebe a análise
    inputs=["analise_tecnica"],
    # Gera o relatório final
    outputs=["relatorio_final"]
)

print("🔧 Steps criados:")
print(f"1️⃣ {step_pesquisa.name} - {step_pesquisa.description}")
print(f"2️⃣ {step_analise.name} - {step_analise.description}")
print(f"3️⃣ {step_relatorio.name} - {step_relatorio.description}")

## 🏭 Montando o Workflow Completo

Agora vamos juntar todos os steps em um workflow. É como montar uma linha de produção!

In [ ]:
# 🏭 Criando o Workflow
workflow_pesquisa = Workflow(
    name="Pesquisa Tecnológica",
    description="Workflow para pesquisar, analisar e reportar sobre tecnologias",
    # Lista dos steps em ordem sequencial
    steps=[
        step_pesquisa,
        step_analise,
        step_relatorio
    ],
    # Debug para ver o que está acontecendo
    debug_mode=True
)

print("🏭 Workflow criado com sucesso!")
print(f"📋 Nome: {workflow_pesquisa.name}")
print(f"🔢 Número de steps: {len(workflow_pesquisa.steps)}")
print(f"📝 Descrição: {workflow_pesquisa.description}")

## 🧪 Testando o Workflow na Prática

Agora vem a parte mais legal! Vamos executar nosso workflow e ver cada step sendo executado em sequência:

In [ ]:
# 🚀 Executando o workflow
print("🚀 Executando workflow para pesquisar sobre 'Machine Learning'...\n")

# Input para o workflow
tecnologia_pesquisar = "Machine Learning"

# Executar o workflow
resultado = workflow_pesquisa.run(
    input={"tecnologia": tecnologia_pesquisar}
)

print("\n" + "="*50)
print("✅ WORKFLOW CONCLUÍDO!")
print("="*50)
print(f"📊 Status: {resultado.status}")
print(f"⏱️ Duração: {resultado.metrics.duration if resultado.metrics else 'N/A'} segundos")
print(f"🎯 Resultado final disponível: {'Sim' if resultado.content else 'Não'}")

In [ ]:
# 📋 Vamos ver o resultado final
if resultado.content:
    print("📄 RELATÓRIO FINAL:")
    print("="*80)
    print(resultado.content)
else:
    print("❌ Nenhum conteúdo foi gerado")

## 📊 Analisando os Resultados do Workflow

Vamos extrair informações detalhadas sobre como o workflow foi executado:

In [ ]:
# 📈 Análise detalhada dos resultados
print("🔍 ANÁLISE DETALHADA DO WORKFLOW\n")

# Informações gerais
print(f"🆔 ID da execução: {resultado.run_id}")
print(f"📅 Workflow: {workflow_pesquisa.name}")
print(f"🔄 Status: {resultado.status}")

# Métricas se disponíveis
if resultado.metrics:
    print(f"\n📊 MÉTRICAS:")
    print(f"⏱️ Tempo total: {resultado.metrics.duration} segundos")
    if hasattr(resultado.metrics, 'total_tokens'):
        print(f"🎯 Tokens utilizados: {resultado.metrics.total_tokens}")

# Informações dos steps executados
print(f"\n🔧 STEPS EXECUTADOS:")
for i, step in enumerate(workflow_pesquisa.steps, 1):
    print(f"{i}️⃣ {step.name}: {step.description}")
    print(f"   👤 Agente: {step.agent.name}")
    print(f"   📥 Inputs: {step.inputs}")
    print(f"   📤 Outputs: {step.outputs}\n")

## 🎯 Workflow com Diferentes Tipos de Steps

Vamos criar um exemplo mais avançado mostrando diferentes tipos de steps e como eles se conectam:

In [ ]:
# 🎯 Criando agentes especializados para um workflow de análise de mercado

# Agente para buscar notícias
news_agent = Agent(
    name="News Hunter",
    model=Gemini(id="gemini-2.0-flash"),
    role="Caçador de notícias tech",
    instructions=[
        "Busque as notícias mais recentes e relevantes",
        "Foque em informações de fontes confiáveis",
        "Resuma os pontos principais de cada notícia"
    ],
    tools=[HackerNewsTools()],
    markdown=True
)

# Agente para análise de tendências
trend_agent = Agent(
    name="Trend Analyzer",
    model=Gemini(id="gemini-2.0-flash"),
    role="Analista de tendências",
    instructions=[
        "Identifique padrões e tendências nas notícias",
        "Faça previsões baseadas nos dados",
        "Destaque oportunidades e riscos"
    ],
    markdown=True
)

# Agente para recomendações estratégicas
strategy_agent = Agent(
    name="Strategy Advisor",
    model=Gemini(id="gemini-2.0-flash"),
    role="Consultor estratégico",
    instructions=[
        "Crie recomendações estratégicas práticas",
        "Considere diferentes cenários",
        "Priorize ações por impacto e viabilidade"
    ],
    markdown=True
)

print("👥 Agentes do workflow de análise de mercado criados!")

In [ ]:
# 🔄 Criando steps mais elaborados

# Step 1: Coleta de notícias
step_noticias = Step(
    name="coleta_noticias",
    description="Coleta as principais notícias sobre o tópico",
    agent=news_agent,
    inputs=["topico"],
    outputs=["noticias_coletadas"]
)

# Step 2: Análise de tendências
step_tendencias = Step(
    name="analise_tendencias",
    description="Analisa tendências baseadas nas notícias",
    agent=trend_agent,
    inputs=["noticias_coletadas"],
    outputs=["analise_tendencias"]
)

# Step 3: Recomendações estratégicas
step_estrategia = Step(
    name="recomendacoes_estrategicas",
    description="Gera recomendações estratégicas baseadas na análise",
    agent=strategy_agent,
    inputs=["analise_tendencias"],
    outputs=["recomendacoes_finais"]
)

# Workflow completo
workflow_mercado = Workflow(
    name="Análise de Mercado Tech",
    description="Workflow completo para análise de mercado tecnológico",
    steps=[
        step_noticias,
        step_tendencias,
        step_estrategia
    ],
    debug_mode=True
)

print("🏭 Workflow de análise de mercado criado!")
print(f"📋 Steps: {[step.name for step in workflow_mercado.steps]}")

In [ ]:
# 🚀 Executando o workflow de análise de mercado
print("🚀 Executando análise de mercado para 'Inteligência Artificial'...\n")

resultado_mercado = workflow_mercado.run(
    input={"topico": "Inteligência Artificial"}
)

print("\n" + "="*60)
print("✅ ANÁLISE DE MERCADO CONCLUÍDA!")
print("="*60)
print(f"📊 Status: {resultado_mercado.status}")

if resultado_mercado.content:
    print("\n📋 RECOMENDAÇÕES ESTRATÉGICAS:")
    print("-" * 40)
    print(resultado_mercado.content)

## 📈 Visualizando o Fluxo de Execução

Vamos criar uma visualização para entender melhor como os dados fluem pelo workflow:

In [ ]:
# 📊 Visualização do fluxo de dados
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from matplotlib.patches import FancyBboxPatch, Circle

fig, ax = plt.subplots(figsize=(14, 10))
ax.set_xlim(0, 12)
ax.set_ylim(0, 10)
ax.axis('off')

# Título
ax.text(6, 9.5, 'Fluxo de Dados no Workflow de Análise de Mercado', 
        fontsize=16, fontweight='bold', ha='center')

# Input inicial
input_circle = Circle((1, 7), 0.5, facecolor='lightgreen', edgecolor='green', linewidth=2)
ax.add_patch(input_circle)
ax.text(1, 7, 'INPUT\nTópico', fontsize=9, fontweight='bold', ha='center', va='center')

# Steps com seus dados
steps_data = [
    (3, 7, 'STEP 1\nNews Hunter', 'Notícias\nColetadas', 'lightblue'),
    (6, 7, 'STEP 2\nTrend Analyzer', 'Análise\nTendências', 'lightyellow'),
    (9, 7, 'STEP 3\nStrategy Advisor', 'Recomendações\nFinais', 'lightcoral')
]

for i, (x, y, step_name, output_name, color) in enumerate(steps_data):
    # Step box
    step_box = FancyBboxPatch((x-1, y-1), 2, 2, 
                             boxstyle="round,pad=0.1", 
                             facecolor=color, 
                             edgecolor='gray', linewidth=2)
    ax.add_patch(step_box)
    ax.text(x, y, step_name, fontsize=10, fontweight='bold', ha='center', va='center')
    
    # Output data
    output_box = FancyBboxPatch((x-0.7, y-3), 1.4, 1, 
                               boxstyle="round,pad=0.05", 
                               facecolor='white', 
                               edgecolor='black', linewidth=1)
    ax.add_patch(output_box)
    ax.text(x, y-2.5, output_name, fontsize=8, ha='center', va='center')
    
    # Seta do step para o output
    ax.arrow(x, y-1, 0, -0.8, head_width=0.1, head_length=0.1, 
             fc='black', ec='black', alpha=0.7)

# Setas horizontais conectando os steps
arrow_positions = [(1.5, 7), (4.5, 7), (7.5, 7)]
for x, y in arrow_positions[:-1]:
    ax.arrow(x, y, 1, 0, head_width=0.15, head_length=0.2, 
             fc='blue', ec='blue', linewidth=2)

# Output final
final_circle = Circle((11, 7), 0.5, facecolor='gold', edgecolor='orange', linewidth=2)
ax.add_patch(final_circle)
ax.text(11, 7, 'OUTPUT\nFinal', fontsize=9, fontweight='bold', ha='center', va='center')

# Seta final
ax.arrow(10, 7, 0.5, 0, head_width=0.15, head_length=0.2, 
         fc='orange', ec='orange', linewidth=2)

# Legenda
ax.text(6, 1, '💡 Cada step processa os dados do anterior e gera novos outputs', 
        fontsize=12, fontweight='bold', ha='center',
        bbox=dict(boxstyle="round,pad=0.3", facecolor='lightyellow', alpha=0.8))

plt.tight_layout()
plt.show()

print("📊 Fluxo de dados visualizado!")

## 🔧 Exercício Prático: Criando Seu Próprio Workflow

Agora é sua vez! Vamos criar um workflow para análise de startups:

**Objetivo**: Analisar uma startup em 3 etapas:
1. **Pesquisa**: Coletar informações sobre a startup
2. **Avaliação**: Avaliar modelo de negócio e potencial
3. **Recomendação**: Dar uma recomendação de investimento

In [ ]:
# 🎯 EXERCÍCIO: Complete o código abaixo para criar o workflow de análise de startups

# Agente 1: Pesquisador de Startups
pesquisador_startup = Agent(
    name="Startup Researcher",
    model=Gemini(id="gemini-2.0-flash"),
    role="Pesquisador especializado em startups",
    instructions=[
        "Pesquise informações detalhadas sobre a startup",
        "Foque no modelo de negócio, fundadores e mercado",
        "Colete dados financeiros se disponíveis",
        "Identifique competidores principais"
    ],
    tools=[DuckDuckGoTools()],
    markdown=True
)

# Agente 2: Avaliador de Negócios
avaliador_startup = Agent(
    name="Business Evaluator",
    model=Gemini(id="gemini-2.0-flash"),
    role="Avaliador de modelos de negócio",
    instructions=[
        "Avalie o potencial de crescimento da startup",
        "Analise riscos e oportunidades",
        "Compare com benchmarks do setor",
        "Identifique pontos fortes e fracos"
    ],
    markdown=True
)

# Agente 3: Consultor de Investimentos
consultor_investimento = Agent(
    name="Investment Advisor",
    model=Gemini(id="gemini-2.0-flash"),
    role="Consultor especializado em investimentos",
    instructions=[
        "Faça recomendações de investimento baseadas na análise",
        "Considere diferentes perfis de risco",
        "Sugira estratégias de entrada e saída",
        "Quantifique riscos e retornos esperados"
    ],
    markdown=True
)

print("👥 Agentes para análise de startups criados!")

In [ ]:
# 🏗️ EXERCÍCIO: Crie os steps do workflow

# Complete os steps abaixo:
step_pesquisa_startup = Step(
    name="pesquisa_startup",
    description="Pesquisa informações sobre a startup",
    agent=pesquisador_startup,
    inputs=["nome_startup"],  # Input: nome da startup
    outputs=["dados_startup"]  # Output: dados coletados
)

step_avaliacao_startup = Step(
    name="avaliacao_negocio",
    description="Avalia o potencial de negócio da startup",
    agent=avaliador_startup,
    inputs=["dados_startup"],  # Recebe os dados da pesquisa
    outputs=["avaliacao_negocio"]  # Gera avaliação
)

step_recomendacao_startup = Step(
    name="recomendacao_investimento",
    description="Gera recomendação de investimento",
    agent=consultor_investimento,
    inputs=["avaliacao_negocio"],  # Recebe a avaliação
    outputs=["recomendacao_final"]  # Gera recomendação
)

# Criar o workflow
workflow_startup = Workflow(
    name="Análise de Startups",
    description="Workflow para análise completa de startups",
    steps=[
        step_pesquisa_startup,
        step_avaliacao_startup,
        step_recomendacao_startup
    ],
    debug_mode=True
)

print("🏭 Workflow de análise de startups criado!")
print(f"🔢 Steps configurados: {len(workflow_startup.steps)}")

In [ ]:
# 🚀 EXERCÍCIO: Execute o workflow para uma startup conhecida

startup_para_analisar = "Nubank"  # Você pode trocar por outra startup

print(f"🔍 Analisando a startup: {startup_para_analisar}\n")

# Execute o workflow
resultado_startup = workflow_startup.run(
    input={"nome_startup": startup_para_analisar}
)

print("\n" + "="*60)
print("✅ ANÁLISE DE STARTUP CONCLUÍDA!")
print("="*60)
print(f"📊 Status: {resultado_startup.status}")

if resultado_startup.content:
    print(f"\n💼 RECOMENDAÇÃO DE INVESTIMENTO PARA {startup_para_analisar.upper()}:")
    print("-" * 50)
    print(resultado_startup.content)
else:
    print("❌ Não foi possível gerar a recomendação")

print("\n🎉 Parabéns! Você criou e executou seu primeiro workflow personalizado!")

## 🔍 Monitoramento e Debug de Workflows

Uma das grandes vantagens dos workflows é que podemos acompanhar cada passo e debugar problemas facilmente:

In [ ]:
# 🐛 Função para analisar execução de workflow
def analisar_execucao_workflow(resultado_workflow, workflow_obj):
    """
    Função para analisar detalhadamente a execução de um workflow
    """
    print("🔍 RELATÓRIO DETALHADO DE EXECUÇÃO\n")
    
    # Informações básicas
    print(f"📋 Workflow: {workflow_obj.name}")
    print(f"🆔 Run ID: {resultado_workflow.run_id}")
    print(f"📊 Status: {resultado_workflow.status}")
    
    # Métricas se disponíveis
    if resultado_workflow.metrics:
        print(f"\n⏱️ PERFORMANCE:")
        print(f"   ⏰ Duração total: {resultado_workflow.metrics.duration:.2f}s")
        if hasattr(resultado_workflow.metrics, 'total_tokens'):
            print(f"   🎯 Tokens totais: {resultado_workflow.metrics.total_tokens}")
    
    # Análise por step
    print(f"\n🔧 ANÁLISE POR STEP:")
    for i, step in enumerate(workflow_obj.steps, 1):
        print(f"\n{i}️⃣ STEP: {step.name}")
        print(f"   📝 Descrição: {step.description}")
        print(f"   👤 Agente: {step.agent.name} ({step.agent.role})")
        print(f"   📥 Inputs esperados: {step.inputs}")
        print(f"   📤 Outputs gerados: {step.outputs}")
        
        # Status do step (se disponível)
        print(f"   ✅ Status: Executado")
    
    # Resultado final
    if resultado_workflow.content:
        print(f"\n📄 RESULTADO FINAL:")
        print(f"   📏 Tamanho: {len(str(resultado_workflow.content))} caracteres")
        print(f"   ✅ Conteúdo gerado: Sim")
    else:
        print(f"\n❌ Nenhum resultado final gerado")
    
    print("\n" + "="*50)

# Aplicar a análise ao nosso workflow de startup
analisar_execucao_workflow(resultado_startup, workflow_startup)

## 📊 Comparativo: Workflows vs Teams vs Agentes Individuais

Vamos criar uma comparação visual para entender quando usar cada abordagem:

In [ ]:
# 📊 Tabela comparativa das abordagens
import pandas as pd

comparacao_data = {
    'Aspecto': [
        'Controle do Fluxo',
        'Previsibilidade',
        'Flexibilidade',
        'Debugging',
        'Performance',
        'Complexidade Setup',
        'Colaboração',
        'Casos de Uso Ideais'
    ],
    'Agente Individual': [
        'Alto (interno)',
        'Média',
        'Baixa',
        'Fácil',
        'Rápida',
        'Simples',
        'Nenhuma',
        'Tarefas simples e diretas'
    ],
    'Team': [
        'Médio (líder)',
        'Média',
        'Alta',
        'Médio',
        'Paralela',
        'Médio',
        'Dinâmica',
        'Brainstorming, discussões'
    ],
    'Workflow': [
        'Total (você)',
        'Alta',
        'Baixa',
        'Fácil',
        'Sequencial',
        'Complexo',
        'Estruturada',
        'Processos definidos'
    ]
}

df_comparacao = pd.DataFrame(comparacao_data)
print("📋 COMPARATIVO: AGENTES vs TEAMS vs WORKFLOWS\n")
print(df_comparacao.to_string(index=False))

print("\n🎯 QUANDO USAR CADA ABORDAGEM:\n")
print("🤖 AGENTE INDIVIDUAL:")
print("   • Tarefa simples e bem definida")
print("   • Não precisa de múltiplas perspectivas")
print("   • Resposta rápida necessária\n")

print("👥 TEAM:")
print("   • Precisa de diferentes especialidades")
print("   • Brainstorming e criatividade")
print("   • Problema sem solução óbvia\n")

print("🔄 WORKFLOW:")
print("   • Processo bem definido")
print("   • Precisa de controle total")
print("   • Auditoria e compliance importantes")
print("   • Processamento em lote")

## 🚀 Casos de Uso Avançados: Workflows na Prática

Vamos ver alguns exemplos reais onde workflows são a melhor escolha:

In [ ]:
# 💼 Exemplos de casos de uso para workflows

casos_uso = {
    "📝 Análise de Documentos": {
        "steps": ["Extração", "Classificação", "Análise", "Relatório"],
        "beneficios": ["Processamento padronizado", "Auditoria completa", "Escalabilidade"]
    },
    "🏭 Pipeline de Dados": {
        "steps": ["Coleta", "Limpeza", "Transformação", "Análise", "Visualização"],
        "beneficios": ["Qualidade garantida", "Reproducibilidade", "Monitoramento"]
    },
    "🛡️ Análise de Segurança": {
        "steps": ["Scan", "Análise de Vulnerabilidades", "Classificação de Riscos", "Recomendações"],
        "beneficios": ["Padronização", "Compliance", "Rastreabilidade"]
    },
    "📊 Relatórios Executivos": {
        "steps": ["Coleta de Métricas", "Análise de Performance", "Insights", "Formatação"],
        "beneficios": ["Consistência", "Automatização", "Pontualidade"]
    }
}

print("🎯 CASOS DE USO IDEAIS PARA WORKFLOWS:\n")

for caso, detalhes in casos_uso.items():
    print(f"{caso}")
    print(f"   🔄 Steps: {' → '.join(detalhes['steps'])}")
    print(f"   ✅ Benefícios: {', '.join(detalhes['beneficios'])}")
    print()

print("💡 DICA: Workflows são perfeitos quando você precisa de:")
print("   • Controle total sobre o processo")
print("   • Auditoria de cada passo")
print("   • Processamento em lote")
print("   • Compliance com regulamentações")

## 🎓 Exercício Final: Workflow de E-commerce

Vamos criar um workflow mais complexo para análise de produtos de e-commerce:

In [ ]:
# 🛒 DESAFIO FINAL: Crie um workflow para análise de produtos de e-commerce

print("🎯 DESAFIO FINAL: Workflow de Análise de E-commerce\n")
print("📋 Objetivo: Criar um workflow que analisa um produto e:")
print("   1. Pesquisa informações do produto")
print("   2. Analisa reviews e avaliações")
print("   3. Compara com concorrentes")
print("   4. Gera recomendações de otimização\n")

# Template para o workflow (você pode completar)
template_workflow = """
# Agentes necessários:
# 1. Product Researcher - pesquisa dados do produto
# 2. Review Analyzer - analisa feedback dos clientes  
# 3. Competitor Analyst - analisa concorrência
# 4. Optimization Advisor - gera recomendações

# Steps do workflow:
# 1. product_research -> dados_produto
# 2. review_analysis -> analise_reviews  
# 3. competitor_analysis -> analise_concorrencia
# 4. optimization_recommendations -> recomendacoes_otimizacao
"""

print("💡 Template do workflow:")
print(template_workflow)

print("\n🚀 Tente implementar este workflow usando o que aprendemos!")
print("\nAlgumas sugestões de produtos para testar:")
print("   • iPhone 15")
print("   • Tesla Model 3")
print("   • MacBook Pro M3")
print("   • PlayStation 5")

## 📚 Resumo do Módulo: Workflows Básicos

Liiindo! Chegamos ao final de mais um módulo! 🎉

### 🧠 O que aprendemos hoje:

1. **Conceitos Fundamentais**:
   - Workflows são fluxos sequenciais controlados
   - Diferente de Teams (paralelo) e Agentes individuais
   - Cada Step tem um agente especializado

2. **Componentes Essenciais**:
   - `Step`: Unidade básica de trabalho
   - `Workflow`: Orquestrador dos steps
   - `Agent`: Executor especializado de cada step

3. **Implementação Prática**:
   - Como criar steps sequenciais
   - Conexão de inputs e outputs
   - Debug e monitoramento

4. **Casos de Uso Reais**:
   - Análise de mercado
   - Pesquisa tecnológica
   - Análise de startups

### 🎯 Quando usar Workflows:

✅ **Use quando:**
- Processo bem definido
- Precisa de controle total
- Auditoria é importante
- Processamento sequencial faz sentido

❌ **Não use quando:**
- Precisa de flexibilidade máxima
- Brainstorming ou criatividade
- Processo muito simples

### 🚀 Preparação para o próximo módulo:

No **Módulo 10** vamos aprender sobre **Workflows Avançados**:
- Fluxos condicionais (if/else)
- Loops e iterações
- Workflows paralelos
- Tratamento de erros
- Workflows complexos

### 💡 Dica final:

Workflows são como receitas de bolo - quanto mais específica a receita, melhor o resultado! No próximo módulo vamos aprender a fazer "receitas inteligentes" que se adaptam às situações.

**Bora para o próximo nível!** 🚀

![](https://s3.us-east-1.amazonaws.com/turing.education/books/imagens/agno-2.0-framework--modulo-09_img_01.png)